In [384]:
import pandas as pd
import seaborn as sns
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import KFold, train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [385]:
train_df = pd.read_csv('./player_data.csv', header=None, skiprows=[0])
label_df = pd.read_csv('./y_summary_data.csv')
vote_df = pd.read_csv('./vote_data.csv')
train_df['label'] = pd.Series([0] * train_df.shape[0])
train_df['vote1'] = pd.Series([0] * train_df.shape[0])
train_df['vote2'] = pd.Series([0] * train_df.shape[0])

In [386]:
train_df[8] = train_df[8].apply(lambda x : float(x.split('%')[0]))
train_df[11] = train_df[11].apply(lambda x : float(x.split('%')[0]))
train_df[14] = train_df[14].apply(lambda x : float(x.split('%')[0]))

In [387]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23027 entries, 0 to 23026
Data columns (total 32 columns):
0        23027 non-null int64
1        23027 non-null int64
2        23027 non-null object
3        23027 non-null object
4        23027 non-null object
5        23027 non-null int64
6        23027 non-null int64
7        23027 non-null float64
8        23027 non-null float64
9        23027 non-null float64
10       23027 non-null float64
11       23027 non-null float64
12       23027 non-null float64
13       23027 non-null float64
14       23027 non-null float64
15       23027 non-null float64
16       23027 non-null float64
17       23027 non-null float64
18       23027 non-null float64
19       23027 non-null float64
20       23027 non-null float64
21       23027 non-null float64
22       23027 non-null float64
23       23027 non-null float64
24       23027 non-null float64
25       23027 non-null float64
26       23027 non-null int64
27       23027 non-null int64
28       2

In [388]:
def add_season(data):
    index = []
    for x in data.iloc[:, 3]:
        index = index + [int(x.split('-')[0].strip())]
        data['season'] = pd.Series(index)

add_season(train_df)

In [389]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23027 entries, 0 to 23026
Data columns (total 33 columns):
0         23027 non-null int64
1         23027 non-null int64
2         23027 non-null object
3         23027 non-null object
4         23027 non-null object
5         23027 non-null int64
6         23027 non-null int64
7         23027 non-null float64
8         23027 non-null float64
9         23027 non-null float64
10        23027 non-null float64
11        23027 non-null float64
12        23027 non-null float64
13        23027 non-null float64
14        23027 non-null float64
15        23027 non-null float64
16        23027 non-null float64
17        23027 non-null float64
18        23027 non-null float64
19        23027 non-null float64
20        23027 non-null float64
21        23027 non-null float64
22        23027 non-null float64
23        23027 non-null float64
24        23027 non-null float64
25        23027 non-null float64
26        23027 non-null int64
27        230

In [390]:
label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 24 columns):
赛季      74 non-null object
球员      74 non-null object
联盟      74 non-null object
出场      74 non-null object
首发      38 non-null object
时间      74 non-null object
投篮      74 non-null object
命中      74 non-null object
出手      74 non-null object
三分      49 non-null object
命中.1    50 non-null object
出手.1    50 non-null object
罚球      74 non-null object
命中.2    74 non-null object
出手.2    74 non-null object
篮板      74 non-null object
前场      56 non-null object
后场      56 non-null object
助攻      74 non-null object
抢断      51 non-null object
盖帽      51 non-null object
失误      52 non-null object
犯规      74 non-null object
得分      74 non-null object
dtypes: object(24)
memory usage: 14.1+ KB


In [391]:
vote.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 26 columns):
Unnamed: 0    929 non-null int64
0             929 non-null object
1             929 non-null object
2             929 non-null object
3             929 non-null object
4             929 non-null int64
5             929 non-null object
6             929 non-null float64
7             929 non-null object
8             929 non-null float64
9             929 non-null float64
10            929 non-null object
11            929 non-null object
12            929 non-null object
13            929 non-null object
14            929 non-null float64
15            929 non-null float64
16            929 non-null float64
17            929 non-null object
18            929 non-null object
19            929 non-null float64
20            929 non-null object
21            929 non-null object
22            929 non-null object
23            929 non-null float64
24            929 non-null float64
dtyp

In [392]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,18,19,20,21,22,23,24,25,26,27,28,label,vote1,vote2,season
0,0,1,Allonzo Trier,18-19,尼克斯,64,3,22.8,44.8,3.6,8.1,39.4,0.8,2.1,80.3,...,0.5,2.6,1.9,0.4,0.2,1.8,1.8,10.9,13,51,6.8,0,0,0,18
1,1,2,Amile Jefferson,18-19,魔术,12,0,5.7,62.5,0.8,1.3,0.0,0.0,0.0,87.5,...,0.5,1.3,0.3,0.3,0.3,0.1,0.7,2.3,9,3,2.6,0,0,0,18
2,2,3,Angel Delgado,18-19,快船,2,0,7.5,20.0,0.5,2.5,0.0,0.0,0.0,50.0,...,1.0,1.0,0.0,0.5,0.0,0.0,1.0,1.5,0,2,0.9,0,0,0,18
3,3,4,Billy Garrett,18-19,尼克斯,4,0,16.0,40.7,2.8,6.8,0.0,0.0,2.0,100.0,...,0.0,0.8,1.8,0.3,0.3,0.5,0.8,6.5,1,3,4.0,0,0,0,18
4,4,5,Bonzie Colson,18-19,雄鹿,8,2,12.3,33.3,1.6,4.9,23.8,0.6,2.6,88.9,...,1.0,2.8,0.4,0.6,0.1,0.4,0.4,4.9,6,2,4.0,0,0,0,18


In [393]:
test_df = train_df[train_df[3] == '18-19']

In [394]:
test_df[test_df[0] == 274]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,18,19,20,21,22,23,24,25,26,27,28,label,vote1,vote2,season
274,274,275,扬尼斯-阿德托昆博,18-19,雄鹿,72,72,32.8,57.8,10.0,17.3,25.6,0.7,2.8,72.9,...,2.2,10.3,5.9,1.3,1.5,3.7,3.2,27.7,56,16,24.6,0,0,0,18


In [395]:
train_df = train_df[train_df[3] != '18-19']

In [396]:
train_df.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,18,19,20,21,22,23,24,25,26,27,28,label,vote1,vote2,season
23022,23022,23023,鲍伯-福特,46-47,反叛者,51,0,0.0,29.5,2.8,9.4,0.0,0.0,0.0,57.5,...,0.0,0.0,0.6,0.0,0.0,0.0,1.9,6.7,0,0,-1.1,0,0,0,46
23023,23023,23024,鲍伯-菲茨杰拉德,46-47,0,60,0,0.0,19.3,1.2,6.0,0.0,0.0,0.0,62.3,...,0.0,0.0,0.6,0.0,0.0,0.0,2.6,3.7,0,0,-2.7,0,0,0,46
23024,23024,23025,鲍伯-迪勒,46-47,猎鹰,57,0,0.0,19.7,1.9,9.9,0.0,0.0,0.0,66.7,...,0.0,0.0,0.7,0.0,0.0,0.0,1.6,5.2,0,0,-3.1,0,0,0,46
23025,23025,23026,鲍伯-道尔,46-47,轰炸机,60,0,0.0,25.3,3.2,12.8,0.0,0.0,0.0,65.0,...,0.0,0.0,0.4,0.0,0.0,0.0,2.8,8.7,0,0,-2.0,0,0,0,46
23026,23026,23027,鲍伯·菲里克,46-47,老首都,55,0,0.0,40.1,6.6,16.5,0.0,0.0,0.0,76.2,...,0.0,0.0,1.3,0.0,0.0,0.0,2.6,16.8,0,0,5.6,0,0,0,46


In [397]:
label_df.head()

,赛季,球员,联盟,出场,首发,时间,投篮,命中,出手,三分,命中.1,出手.1,罚球,命中.2,出手.2,篮板,前场,后场,助攻,抢断,盖帽,失误,犯规,得分
0,17-18,詹姆斯-哈登,NBA,72,72,35.4,44.90%,9,20.1,36.70%,3.7,10,85.80%,8.7,10.1,5.4,0.6,4.8,8.8,1.8,0.7,4.4,2.3,30.4
1,16-17,拉塞尔-威斯布鲁克,NBA,81,81,34.6,42.50%,10.2,24,34.30%,2.5,7.2,84.50%,8.8,10.4,10.7,1.7,9,10.4,1.6,0.4,5.4,2.3,31.6
2,15-16,斯蒂芬-库里,NBA,79,79,34.2,50.40%,10.2,20.2,45.40%,5.1,11.2,90.80%,4.6,5.1,5.4,0.9,4.6,6.7,2.1,0.2,3.3,2,30.1
3,14-15,斯蒂芬-库里,NBA,80,80,32.7,48.70%,8.2,16.8,44.30%,3.6,8.1,91.40%,3.9,4.2,4.3,0.7,3.6,7.7,2,0.2,3.1,2,23.8
4,13-14,凯文-杜兰特,NBA,81,81,38.5,50.30%,10.5,20.8,39.10%,2.4,6.1,87.30%,8.7,9.9,7.4,0.7,6.7,5.5,1.3,0.7,3.5,2.1,32


In [398]:
cnt = 0
for x in np.arange(train_df.shape[0]):
    if train_df.iloc[x,:][2] == label_df['球员'][cnt] and train_df.iloc[x ,3] == label_df['赛季'][cnt]:
        train_df.iloc[x,29] = 1.
        cnt += 1
        print(train_df.iloc[x, 2],train_df.iloc[x,29], train_df.iloc[x,3])

詹姆斯-哈登 1.0 17-18
拉塞尔-威斯布鲁克 1.0 16-17
斯蒂芬-库里 1.0 15-16
斯蒂芬-库里 1.0 14-15
凯文-杜兰特 1.0 13-14
勒布朗-詹姆斯 1.0 12-13
勒布朗-詹姆斯 1.0 11-12
德里克-罗斯 1.0 10-11
勒布朗-詹姆斯 1.0 09-10
勒布朗-詹姆斯 1.0 08-09
科比-布莱恩特 1.0 07-08
德克-诺维茨基 1.0 06-07
史蒂夫-纳什 1.0 05-06
史蒂夫-纳什 1.0 04-05
凯文-加内特 1.0 03-04
蒂姆-邓肯 1.0 02-03
蒂姆-邓肯 1.0 01-02
阿伦-艾弗森 1.0 00-01
沙奎尔-奥尼尔 1.0 99-00
卡尔-马龙 1.0 98-99
迈克尔-乔丹 1.0 97-98
卡尔-马龙 1.0 96-97
迈克尔-乔丹 1.0 95-96
大卫-罗宾逊 1.0 94-95
哈基姆-奥拉朱旺 1.0 93-94
查尔斯-巴克利 1.0 92-93
迈克尔-乔丹 1.0 91-92
迈克尔-乔丹 1.0 90-91
埃尔文-约翰逊 1.0 89-90
埃尔文-约翰逊 1.0 88-89
迈克尔-乔丹 1.0 87-88
埃尔文-约翰逊 1.0 86-87
拉里-伯德 1.0 85-86
拉里-伯德 1.0 84-85
拉里-伯德 1.0 83-84
摩西-马龙 1.0 82-83
摩西-马龙 1.0 81-82
朱利叶斯-欧文 1.0 80-81
卡里姆-贾巴尔 1.0 79-80
摩西-马龙 1.0 78-79
比尔-沃顿 1.0 77-78
卡里姆-贾巴尔 1.0 76-77
卡里姆-贾巴尔 1.0 75-76
鲍伯-麦克阿杜 1.0 74-75
卡里姆-贾巴尔 1.0 73-74
戴夫-考文斯 1.0 72-73
卡里姆-贾巴尔 1.0 71-72
卡里姆-贾巴尔 1.0 70-71
威利斯-里德 1.0 69-70
韦斯-昂塞尔德 1.0 68-69
威尔特-张伯伦 1.0 67-68
威尔特-张伯伦 1.0 66-67
威尔特-张伯伦 1.0 65-66
比尔-拉塞尔 1.0 64-65
奥斯卡-罗波特森 1.0 63-64
比尔-拉塞尔 1.0 62-63
比尔-拉塞尔 1.0 61-62
比尔-拉塞尔 1.0 6

In [399]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,18,19,20,21,22,23,24,25,26,27,28,label,vote1,vote2,season
528,528,529,Alec Peters,17-18,太阳,20,0,11.2,37.8,1.4,3.7,31.0,0.9,2.9,80.0,...,0.4,1.5,0.6,0.1,0.1,0.2,0.7,4.1,2,18,2.9,0.0,0,0,17
529,529,530,C.J. Miles,17-18,猛龙,70,3,19.1,37.9,3.2,8.6,36.1,2.3,6.5,83.5,...,0.3,1.9,0.8,0.5,0.3,0.6,1.9,10.0,50,20,5.9,0.0,0,0,17
530,530,531,C.J. 威廉姆斯,17-18,快船,38,18,18.5,44.2,2.3,5.2,28.2,0.6,2.1,81.3,...,0.3,1.2,1.1,0.8,0.3,0.5,1.7,5.5,17,21,4.0,0.0,0,0,17
531,531,532,C.J.迈克鲁姆,17-18,开拓者,81,81,36.1,44.3,8.2,18.6,39.7,2.3,5.9,83.6,...,0.7,3.3,3.4,1.0,0.4,1.9,2.1,21.4,48,33,13.8,0.0,0,0,17
532,532,533,D.J.威尔森,17-18,雄鹿,22,0,3.3,56.3,0.4,0.7,40.0,0.1,0.2,50.0,...,0.1,0.4,0.1,0.1,0.0,0.2,0.3,1.0,12,10,0.7,0.0,0,0,17


In [400]:
vote_df.tail(12)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
917,917,56-57,01.18%,01.18%,哈里-盖拉汀,72,,27.0,40.6%,4.6,11.3,,,,80.0%,5.8,7.2,10.1,,,1.2,,,,2.8,15.0
918,918,56-57,01.18%,01.18%,迈尔-哈特金斯,72,,36.8,38.7%,5.1,13.2,,,,73.8%,2.1,2.9,7.9,,,2.9,,,,2.5,12.4
919,919,56-57,01.18%,01.18%,Kenny Sears,72,,34.9,41.8%,4.8,11.4,,,,79.0%,5.3,6.7,8.5,,,1.4,,,,3.1,14.8
920,920,56-57,01.18%,01.18%,杰克-特威曼,72,,32.5,43.9%,6.2,14.2,,,,76.0%,3.8,5.0,4.9,,,1.7,,,,3.5,16.3
921,921,55-56,43.21%,43.21%,鲍勃-佩蒂特,72,,38.8,42.9%,9.0,20.9,,,,73.6%,7.7,10.5,16.2,,,2.6,,,,2.8,25.7
922,922,55-56,25.93%,25.93%,保罗-阿里金,72,,37.8,44.8%,8.6,19.1,,,,81.0%,7.0,8.7,7.5,,,2.6,,,,3.9,24.2
923,923,55-56,12.35%,12.35%,鲍勃-库西,72,,38.4,36.0%,6.1,17.0,,,,84.4%,6.6,7.8,6.8,,,8.9,,,,2.9,18.8
924,924,55-56,11.11%,11.11%,迈尔-哈特金斯,66,,33.9,42.5%,4.9,11.6,,,,64.3%,2.2,3.3,7.5,,,2.7,,,,2.5,12.0
925,925,55-56,02.47%,02.47%,多尔夫-谢伊斯,72,,35.0,38.7%,6.5,16.7,,,,85.8%,7.5,8.8,12.4,,,2.8,,,,3.5,20.4
926,926,55-56,02.47%,02.47%,比尔-沙尔曼,72,,37.5,43.8%,7.5,17.1,,,,86.7%,5.0,5.7,3.6,,,4.7,,,,2.7,19.9


In [401]:
cnt = 0
for x in np.arange(train_df.shape[0]):
    if train_df.iloc[x,:][2] in vote_df.iloc[:, 4].values:
        temp = vote_df.iloc[:, :][vote_df.iloc[:, 4] == train_df.iloc[x,:][2]]
        for y in np.arange(temp.shape[0]):          
            if temp.iloc[y, 1] == train_df.iloc[x ,3]:
                train_df.iloc[x,30] = float(temp.iloc[y, 2].split('%')[0])
                train_df.iloc[x,31] = float(temp.iloc[y, 3].split('%')[0])
                cnt += 1
                print(train_df.iloc[x, 2],train_df.iloc[x,30], train_df.iloc[x,3], train_df.iloc[x,4])

乔尔-恩比德 0.0 17-18  76人
凯文-杜兰特 0.0 17-18 勇士
勒布朗-詹姆斯 14.85 17-18 骑士
吉米-巴特勒 0.0 17-18 森林狼
安东尼-戴维斯 0.0 17-18 鹈鹕
德玛尔-德罗赞 0.0 17-18 猛龙
扬尼斯-阿德托昆博 0.0 17-18 雄鹿
拉塞尔-威斯布鲁克 0.0 17-18 雷霆
拉玛库斯-阿尔德里奇 0.0 17-18 马刺
斯蒂芬-库里 0.0 17-18 勇士
维克多-奥拉迪波 0.0 17-18 步行者
詹姆斯-哈登 85.15 17-18 火箭
达米安-利拉德 0.0 17-18 开拓者
以赛亚-托马斯 0.0 16-17 凯尔特
凯文-杜兰特 0.0 16-17 勇士
勒布朗-詹姆斯 0.99 16-17 骑士
安东尼-戴维斯 0.0 16-17 鹈鹕
德玛尔-德罗赞 0.0 16-17 猛龙
扬尼斯-阿德托昆博 0.0 16-17 雄鹿
拉塞尔-威斯布鲁克 68.32 16-17 雷霆
斯蒂芬-库里 0.0 16-17 勇士
科怀-伦纳德 8.91 16-17 马刺
约翰-沃尔 0.0 16-17 奇才
詹姆斯-哈登 21.78 16-17 火箭
克里斯-保罗 0.0 15-16 快船
凯尔-洛瑞 0.0 15-16 猛龙
凯文-杜兰特 0.0 15-16 雷霆
勒布朗-詹姆斯 0.0 15-16 骑士
德雷蒙德-格林  0.0 15-16 勇士
拉塞尔-威斯布鲁克 0.0 15-16 雷霆
斯蒂芬-库里 100.0 15-16 勇士
科怀-伦纳德 0.0 15-16 马刺
詹姆斯-哈登 0.0 15-16 火箭
达米安-利拉德 0.0 15-16 开拓者
克莱-汤普森 0.0 14-15 勇士
克里斯-保罗 0.0 14-15 快船
勒布朗-詹姆斯 3.85 14-15 骑士
安东尼-戴维斯 0.0 14-15 鹈鹕
布雷克-格里芬 0.0 14-15 快船
拉塞尔-威斯布鲁克 0.0 14-15 雷霆
拉玛库斯-阿尔德里奇 0.0 14-15 开拓者
斯蒂芬-库里 76.92 14-15 勇士
科怀-伦纳德 0.0 14-15 马刺
蒂姆-邓肯 0.0 14-15 马刺
詹姆斯-哈登 19.23 14-15 火箭
马克-加索尔 0.0 14-15 灰熊
乔金-诺阿 0.0 13-14

帕特里克-尤因 4.08 92-93 尼克斯
查尔斯-巴克利 60.2 92-93 太阳
沙奎尔-奥尼尔 0.0 92-93 魔术
约翰-斯托克顿 0.0 92-93 爵士
肖恩-坎普 0.0 92-93 超音速
迈克尔-乔丹 13.27 92-93 公牛
马克-普莱斯 0.0 92-93 骑士
丹尼-曼宁 0.0 91-92 快船
丹尼斯-罗德曼 0.0 91-92 活塞
克莱德-德雷克斯勒 12.5 91-92 开拓者
克里斯-穆林 0.0 91-92 勇士
凯文-约翰逊 0.0 91-92 太阳
卡尔-马龙 1.04 91-92 爵士
大卫-罗宾逊 2.08 91-92 马刺
布拉德-多赫迪 0.0 91-92 骑士
帕特里克-尤因 0.0 91-92 尼克斯
德特雷夫-施拉姆夫 0.0 91-92 步行者
拉里-伯德 0.0 91-92 凯尔特
斯科蒂-皮蓬 1.04 91-92 公牛
查尔斯-巴克利 0.0 91-92  76人
约翰-斯托克顿 0.0 91-92 爵士
蒂姆-哈达威 0.0 91-92 勇士
迈克尔-乔丹 83.33 91-92 公牛
马克-普莱斯 0.0 91-92 骑士
乔-杜马斯 0.0 90-91 活塞
伊塞亚-托马斯 0.0 90-91 活塞
伯纳德-金 0.0 90-91 子弹
克莱德-德雷克斯勒 1.04 90-91 开拓者
凯文-约翰逊 0.0 90-91 太阳
凯文-麦克海尔 0.0 90-91 凯尔特
卡尔-马龙 0.0 90-91 爵士
哈基姆-奥拉朱旺 0.0 90-91 火箭
埃尔文-约翰逊 10.42 90-91 湖人
多米尼克-威尔金斯 0.0 90-91 老鹰
大卫-罗宾逊 6.25 90-91 马刺
帕特里克-尤因 0.0 90-91 尼克斯
拉里-伯德 0.0 90-91 凯尔特
查尔斯-巴克利 2.08 90-91  76人
特里-伯特 0.0 90-91 开拓者
约翰-斯托克顿 0.0 90-91 爵士
罗波特-帕里什 0.0 90-91 凯尔特
肯尼-史密斯 0.0 90-91 火箭
蒂姆-哈达威 0.0 90-91 勇士
迈克尔-乔丹 80.21 90-91 公牛
乔-杜马斯 0.0 89-90 活塞
伊塞亚-托马斯 0.0 89-90 活塞
克莱德-德雷克斯勒 0.0 89-90 开拓者
卡尔

卡里姆-贾巴尔 49.39 71-72 雄鹿
威尔特-张伯伦 21.95 71-72 湖人
斯宾瑟-海伍德 1.22 71-72 超音速
杰里-韦斯特 26.83 71-72 湖人
约翰-哈福利切克 0.61 71-72 凯尔特
卡里姆-贾巴尔 86.93 70-71 雄鹿
奥斯卡-罗波特森 3.27 70-71 雄鹿
威利斯-里德 3.27 70-71 尼克斯
戴夫-宾 3.92 70-71 活塞
杰里-韦斯特 2.61 70-71 湖人
卡里姆-贾巴尔 20.0 69-70 雄鹿
威利斯-里德 40.67 69-70 尼克斯
康尼尔-霍金斯 1.33 69-70 太阳
杰里-韦斯特 34.0 69-70 湖人
比利-康宁汉姆 1.33 69-70  76人
沃尔特-弗雷泽 2.67 69-70 尼克斯
埃尔金-贝勒 5.83 68-69 湖人
威利斯-里德 17.48 68-69 尼克斯
比利-康宁汉姆 14.56 68-69  76人
比尔-拉塞尔 10.68 68-69 凯尔特
韦斯-昂塞尔德 51.46 68-69 子弹
兰尼-威尔肯斯 19.26 67-68 老鹰
埃尔金-贝勒 6.67 67-68 湖人
奥斯卡-罗波特森 2.22 67-68 皇家
威尔特-张伯伦 65.19 67-68  76人
戴夫-宾 6.67 67-68 活塞
内特-瑟蒙德 14.29 66-67 勇士
奥斯卡-罗波特森 2.86 66-67 皇家
威尔特-张伯伦 76.19 66-67  76人
比尔-拉塞尔 4.76 66-67 凯尔特
里克-巴里 1.9 66-67 勇士
奥斯卡-罗波特森 15.73 65-66 皇家
威尔特-张伯伦 53.93 65-66  76人
杰里-卢卡斯 2.25 65-66 皇家
杰里-韦斯特 17.98 65-66 湖人
比尔-拉塞尔 7.87 65-66 凯尔特
萨姆-琼斯 2.25 65-66 凯尔特
奥斯卡-罗波特森 21.21 64-65 皇家
威尔特-张伯伦 1.01 64-65 0
杰里-韦斯特 20.2 64-65 湖人
比尔-拉塞尔 53.54 64-65 凯尔特
萨姆-琼斯 4.04 64-65 凯尔特
埃尔金-贝勒 0.0 63-64 湖人
奥斯卡-罗波特森 62.5 63-64 皇家
威尔特-张伯伦 19.79 63-

In [402]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 528 entries, 0 to 527
Data columns (total 33 columns):
0         528 non-null int64
1         528 non-null int64
2         528 non-null object
3         528 non-null object
4         528 non-null object
5         528 non-null int64
6         528 non-null int64
7         528 non-null float64
8         528 non-null float64
9         528 non-null float64
10        528 non-null float64
11        528 non-null float64
12        528 non-null float64
13        528 non-null float64
14        528 non-null float64
15        528 non-null float64
16        528 non-null float64
17        528 non-null float64
18        528 non-null float64
19        528 non-null float64
20        528 non-null float64
21        528 non-null float64
22        528 non-null float64
23        528 non-null float64
24        528 non-null float64
25        528 non-null float64
26        528 non-null int64
27        528 non-null int64
28        528 non-null float64
label     5

In [460]:
train_data = train_df.drop([0, 1, 2, 3, 4, 'label', 'vote2', 'vote1'], axis=1).values
train_label = train_df[['vote2', 'vote1']].values

In [461]:
test_data = test_df.drop([0, 1, 2, 3, 4, 'label', 'vote2', 'vote1'], axis=1).values

In [462]:
print(train_data.shape, train_label.shape)
print(test_data.shape)

(22499, 25) (22499, 2)
(528, 25)


In [473]:
class NN(nn.Module):
    def __init__(self, input_):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
        
    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.dropout(x)
        x = F.tanh(self.fc2(x))
#         x = F.dropout(x)
        x = self.fc3(x)
        return x

In [480]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
model = NN(train_data.shape[1]).to(device)
optimzer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [481]:
def train(model, optimzer, criterion, train_data_, train_label_, test_data_, epoch=3000):
    for idx in range(epoch):
        train_data = torch.from_numpy(train_data_.astype(np.float32)).to(device)
        train_label = torch.from_numpy(train_label_.astype(np.float32)).to(device)
#         print(train_data, train_label)
        optimzer.zero_grad()
        y_hat = model(train_data)
#         print(y_hat)
        loss = criterion(y_hat, train_label)
        loss.backward()
        optimzer.step()
#         if idx%100 == 0:
#             print('loss: {}'.format(loss.item()))
    test_data = torch.from_numpy(test_data_.astype(np.float32)).to(device)
    pre = model(test_data)
    pre = pre.to('cpu').detach().numpy()
    print(np.argpartition(pre[:,0], -5)[-5:])
    for x in np.argpartition(pre[:,0], -5)[-5:]:
        print('vote ratio:{}'.format(pre[x,:]))
        print('MVP candidates : {}'.format(str(test_df[test_df[0] == x][2].values)))
#     a = np.argmax(pre[:,0])
#     print('vote ratio:{}'.format(pre[a,:]))
#     print('Final MVP Player is : {}'.format(str(test_df[test_df[0] == a][2].values)))
            
        

In [482]:
train(model, optimzer, criterion, train_data, train_label, test_data)

[204 134 422  62 274]
vote ratio:[6.589641  3.2013338]
MVP candidates : ['尼科拉-约季奇']
vote ratio:[7.137956  4.5967636]
MVP candidates : ['勒布朗-詹姆斯']
vote ratio:[19.941128 17.938374]
MVP candidates : ['詹姆斯-哈登']
vote ratio:[8.864767 3.043383]
MVP candidates : ['乔尔-恩比德']
vote ratio:[31.35448 40.5816 ]
MVP candidates : ['扬尼斯-阿德托昆博']
